<a href="https://colab.research.google.com/github/bimal-bp/Complex_PDF_RAG_Llamaindex/blob/main/COMPLEX_PDF_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# loging to HF
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install llama-index-llms-huggingface llama-index-embeddings-huggingface --quiet
!pip install transformers accelerate bitsandbytes --quiet
!pip install llama-index --quiet
!pip install llama-index-postprocessor-flag-embedding-reranker --quiet
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git --quiet
!pip install llama-parse --quiet
!pip install pyvis --quiet

!pip install "transformers[torch]" "huggingface_hub[inference]" --quiet
! pip install --upgrade llama-index
! pip install llama-index-postprocessor-flag-embedding-reranker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!mkdir -p 'data/'
!wget 'https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/uber_2021.pdf'
!wget 'https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/lyft_2021.pdf'
!wget 'https://s201.q4cdn.com/141608511/files/doc_financials/2022/q4/ca04d49f-caab-436f-9e6f-e8493c8e8515.pdf' -O 'data/nvidia_2021.pdf'

--2024-03-31 14:07:52--  https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/uber_2021.pdf’

data/uber_2021.pdf      [ <=>                ] 156.76K  --.-KB/s    in 0.01s   

2024-03-31 14:07:53 (11.2 MB/s) - ‘data/uber_2021.pdf’ saved [160527]

--2024-03-31 14:07:53--  https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/lyft_2021.pdf’

data/lyft_2021.pdf      [ <=>                ] 156.78K  --.-KB/s    in 0.008s  

2024-03-31 14:07:53 (18.5 MB/s) - ‘data/lyft_2021.pdf’ save

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_parse import LlamaParse

import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

In [6]:
import os
from typing import List,Optional
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)
Settings.llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
    parameters ={"do_sample":False}
)

In [12]:
from llama_parse.base import LlamaParse
import time

# Initialize LlamaParse object
parser = LlamaParse(
    api_key="llx-Jpene0KsYy13zeDYP1kc0HW3VAgNmq33hQ2bXUdwBPUPrCgM",
    result_type="markdown"
)

# Define function to retry parsing with timeout
def parse_with_timeout(file_path, timeout_seconds=138000):
    start_time = time.time()
    job_id = parser.load_data(file_path)
    print(f"Started parsing the file under job_id {job_id}")

    while True:
        try:
            result = parser._get_job_result(job_id, result_type="markdown")
            print("Parsing completed successfully.")
            return result
        except Exception as e:
            if time.time() - start_time > timeout_seconds:
                raise TimeoutError(f"Timeout while parsing the file: {job_id}") from e
            print("Retrying...")
            time.sleep(5)  # Adjust the retry interval as needed

# Attempt to parse the PDF file with a timeout of 1800 seconds (30 minutes)
try:
    lyft_docs = parse_with_timeout('data/lyft_2021.pdf', timeout_seconds=149000)
    nvidia_docs= parse_with_timeout('data/nvidia_2021.pdf',timeout_seconds=149000)
    uber_docs = parse_with_timeout('data/uber_2021.pdf',timeout_seconds=149000)
except TimeoutError as e:
    print(e)


Started parsing the file under job_id efc36a2d-6db5-4419-9eaf-5f9fb93f0afa
...................................................................................................Error while parsing the file 'data/lyft_2021.pdf': Timeout while parsing the file: efc36a2d-6db5-4419-9eaf-5f9fb93f0afa


Exception: Timeout while parsing the file: efc36a2d-6db5-4419-9eaf-5f9fb93f0afa